In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, complete, fcluster
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist
from scipy.special import perm, comb
import random
import copy
%matplotlib inline
#np.set_printoptions(precision=5, suppress=True)


In [ ]:
# 阈值
pccin_min = 0.75
element_limit = 20


In [ ]:
# 数据导入
frame1 = pd.read_csv('./data/1.csv')
#frame2 = copy.deepcopy(frame1)

In [ ]:
# 删除多余数据
del frame1['symbol']
frame1 = frame1.to_numpy()

# 计算总表格皮尔森相关系数
pc = np.corrcoef(frame1)
pc = np.abs(pc)


In [ ]:

# 删除对角线
for i in range(len(pc)):
    pc[i, i] = 0



In [ ]:
#pc = np.row_stack((np.zeros(len(pc)), pc))
#pc = np.column_stack((np.zeros(len(pc)), pc))

In [ ]:
# 深度拷贝，防止影响原矩阵
pc_temp = copy.deepcopy(pc)
pc_find = copy.deepcopy(pc)
pc_find = np.tril(pc_find)

# 聚类分组初始化
cluster = np.empty([10000, 1000], dtype=int)

# 聚类计数
i = 0

In [ ]:
while 1:

    # 求出最大相关系数
    x = np.max(pc_find)

    # 标记矩阵中无剩余元素，循环跳出，将所有聚类结果输出
    if x == 0:
        cluster = cluster[~(cluster == 0).all(1)]
        idx = np.argwhere(np.all(cluster[..., :] == 0, axis=0))
        cluster = np.delete(cluster, idx, axis=1)

        np.savetxt('result1.csv', cluster, delimiter=',', fmt='%d')
        print("done!")
        break

    print("i:", i)

    # 查找最大相关系数的坐标
    max_peer = np.where(pc_temp == np.max(pc_temp))

    # 最大相关系数对写入聚类分组
    cluster[i][0] = max_peer[0][0]
    cluster[i][1] = max_peer[1][0]

    # 在标记矩阵中移除写入聚类分组的数据
    pc_find[max_peer[0][0]] = 0
    pc_find[max_peer[1][0]] = 0
    pc_find[:, max_peer[0][0]] = 0
    pc_find[:, max_peer[1][0]] = 0

    pc_temp[:, max_peer[0][0]] = 0
    pc_temp[:, max_peer[1][0]] = 0

    # 聚类内元素计数，初始为2，PCCin即为相关系数
    times = 2
    pccin = x

    while 1:

        # 初始化
        group = 0

        # 聚类内元素对应相关系数行相加
        for j in range(times):  # j
            group = group + pc_temp[cluster[i, j]]

        # 查找外部相关系数最大的元素
        max_peer_next = np.where(group == np.max(group))[0][0]

        # 判断是否还存在元素
        if max_peer_next == 0:
            break

        # 将该元素加入聚类内
        cluster[i, times] = max_peer_next

        # 在标记矩阵中移除该加入聚类的元素
        pc_find[max_peer_next] = 0
        pc_find[:, max_peer_next] = 0

        pc_temp[:, max_peer_next] = 0

        # 计算聚类内PCCin之和，判断是否跳出
        for j in range(times):
            pccin = pccin + pc[(cluster[i, j]), (cluster[i, times])]

        # 聚类内元素计数加一
        times = times + 1

        # 计算聚类内PCCin均值
        pccin_ave = pccin / comb(times, 2)

        # 聚类内阈值检测，跳出
        if (pccin_ave < pccin_min) and (times >= element_limit):
            break
        if times >= 1000:
            break

    i = i+1
